# 5-3. 트리의 앙상블
- 랜덤 포레스트
- 현시점에서 가장 보편적으로, 성능이 높다고 알려진 알고리즘.
  > `정형 데이터` 에서
  > e.g. 표 구조의 숫자 데이터 (csv) 혹은 엑셀
  > > 정형 데이터와 비정형 데이터
  > >
  - 부트스트랩 샘플링: 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식 \
  `RandomForestRegressor()`

In [1]:
import pandas as pd

df = pd.read_csv('http://bit.ly/wine_csv_data')
data = df[['alcohol', 'sugar', 'pH']]
target = df['class']

In [2]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target =\
train_test_split(data, target)

## `RandomForestClassifier`
#### 랜덤포레스트는 결정트리의 앙상블이기 떄문에 
- DecisionTreeClassifier 가 제공하는 중요한 매개 변수를 모두 제공함.
- 
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [3]:
# 랜덤포레스트 실행
# RandomForestClassifier

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=1)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=1)
print(scores)

# cross_validate는 교차 검증을 수행하여 모델의 성능을 평가하는 함수이다.
# return_train_score=True 옵션을 사용하면 훈련 세트 점수도 함께 제공된다.
# 결과는 fit_time, score_time, test_score, train_score 등을 포함하는 딕셔너리 형태로 반환된다.
#
# 💡 이 코드의 목적은 랜덤 포레스트 모델이 훈련 데이터와 검증 데이터에서 얼마나 잘 작동하는지를 확인하는 것!

{'fit_time': array([0.21650887, 0.21501184, 0.26164246, 0.47878242, 0.29789734]), 'score_time': array([0.0122118 , 0.01175666, 0.01659751, 0.02557254, 0.01661754]), 'test_score': array([0.89230769, 0.88102564, 0.8798768 , 0.86960986, 0.89219713]), 'train_score': array([0.99846035, 0.99820375, 0.99794767, 0.99717804, 0.99820421])}


In [4]:
# 방금 학습한 랜덤포레스트에; feature_importances_ 확인하기

rf.fit(train_input, train_target)
rf.feature_importances_

# 훈련 세트에 훈련한 후 '특성 중요도'를 출력함. |

# 각각 [알코올, 도수, 당도, pH] 였으므로, 결과값을 읽었을 때, 두번째인 '당도'의 중요도가 화이트와 레드를 나누는데에 가장 중요한 것 같다~
# 특성을 랜덤하게 선택했기 때문에 (내가 쓰고 싶은 특성을 선택해서 만들어서) => 과대적합을 줄이고 일반화를 할 수 있다.

array([0.23030179, 0.50673411, 0.26296409])

In [5]:
# 천 개 중에 랜덤하게 백 개를 뽑아 올 수도 있고, 삼십 개만 뽑아 올 수도 있다. 
# 그런 과정에서 한 번도 선택받지 못한 데이터가 있을 수 있다.

# 그 선택 받지 못한 데이터를
# OOB 샘플이라고 부른다.
# 이는 한번도 훈련에 포함되지 않았기 때문에 나중에 테스트에 활용할 수 있다. 

# OOB 점수 출력하기
rf= RandomForestClassifier(oob_score=True, n_jobs=1)
rf.fit(train_input, train_target)
rf.oob_score_

0.889367816091954

## 엑스트라 트리
> 랜덤 포레스트와 매우 비슷하게 동작
> 기본적으로 100개의 결정 트리를 훈련함.
>
> 전체 훈련 세트를 사용 (부트스트램 샘플을 사용하지 않음)
> 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 `무작위로 분할`!
>
- 장점 : 속도가 빠르다~ (최적의 분할을 찾는 데 시간을 소요하지 않아도 되서)

In [6]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=1)

scores= cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=1)
print(scores)

#랜덤포레스트와 비슷한 결과값을 보여줌.
# 우리의 데이터가 많지가 않아서~

{'fit_time': array([0.33855247, 0.32953286, 0.32647657, 0.32715821, 0.33257031]), 'score_time': array([0.03223801, 0.03236914, 0.03131843, 0.03167677, 0.03340745]), 'test_score': array([0.89128205, 0.88102564, 0.88706366, 0.87577002, 0.89117043]), 'train_score': array([0.99846035, 0.99820375, 0.99794767, 0.99743458, 0.99820421])}


## 그래디언트 부스팅 (Gradient boosting)
`from sklearn.ensemble import GradientBoostingClassifier`

> 경사하강법을 적용한 모델
> 나무의 깊이가 깊지 않은 모델
>
> 트리의 오차를 보완하는 방법으로 앙상블 하는 방법
>
장점: 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대 할 수 있다~~~~~~~~~

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_validate

# 모델 객체 생성
# gb = GradientBoostingClassifier(n_estimators=500)
# 심을 나무의 개수를 지정해 줄 수도 있음

gb = GradientBoostingClassifier()


# 교차 검증 실행
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=1)

# 결과 출력
print(scores)


{'fit_time': array([0.39380932, 0.39268279, 0.3894341 , 0.38666368, 0.39117146]), 'score_time': array([0.00482488, 0.00455189, 0.00437999, 0.00475574, 0.00462174]), 'test_score': array([0.87897436, 0.87487179, 0.86755647, 0.85112936, 0.87371663]), 'train_score': array([0.88657942, 0.8860662 , 0.88583889, 0.89533094, 0.88789123])}


---

#### 히스토그램 기반 그레디언트 부스팅 (p.272)
먼저 구간을 나눈다. => 노드를 분할 할 때 최적의 분할을 매우 빠르게 찾을 수 있다. (구간별로 일반화해서)

`HistGradientBoostingClassifier`

In [13]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier()
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=1)
print(scores)

{'fit_time': array([3.0472033 , 3.35590792, 2.72461677, 2.70470595, 2.68993855]), 'score_time': array([0.02754188, 0.02606964, 0.02246785, 0.02282572, 0.02326059]), 'test_score': array([0.87282051, 0.88      , 0.86858316, 0.86036961, 0.87268994]), 'train_score': array([0.93174237, 0.93045933, 0.93355567, 0.93355567, 0.92560287])}


In [15]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
scores = permutation_importance(hgb, train_input, train_target)
print(scores.importances_mean)

# 당도 점수를 섞었을 때, 수치가 많이 낮아지네??

[0.08752053 0.24860427 0.07610837]


#### 사이킷런 말고도 그래디언트 부스팅 알고리즘을 구현한 라이브러리가 여럿있음.
> XGBoost 등 -> 알고 싶다면 검색해서 ~